In [3]:


def findHomology(sequence1, sequence2): #Returns largest homologous region between the 2 sequences
    m = [[0] * (1+len(sequence2))
    for i in range(1+len(sequence1))]
        longest, x_longest = 0, 0
        for x in range(1, 1 + len(sequence2)):
            for y in range(1, 1 + len(sequence2)):
                if sequence1[x - 1] == sequence2[y - 1]:
                    m[x][y] = m[x - 1][y - 1] + 1
                    if m[x][y] > longest:
                        longest = m[x][y]
                        x_longest = x
                else:
                    m[x][y] = 0
    return sequence1[x_longest - longest: x_longest]
        
def choosePrimers(insert, upstreamBackbone, downstreamBackbone): #Make primers for an insert to amplify it such that it can fit in a 
    forwardPrimer = insert[:20]
    reversePrimer = reverseComplement(insert[len(insert) - 20: len(insert)])
    getTm
    
def reverseComplement(sequence):
    reverseComplement = ""
    for base in sequence:
        if base == "a":
            reverseComplement = reverseComplement + "t"
        elif base == "t":
            reverseComplement = reverseComplement + "a"
        elif base == "g":
            reverseComplement = reverseComplement + "c"
        elif base == "c":
            reverseComplement = reverseComplement + "g"
        else:
            print ("error")
    return (reverseComplement)
def getTm(sequence):
    composition = getComposititon(sequence)
    #Tm= 64.9 +41*(yG+zC-16.4)/(wA+xT+yG+zC) 
    Tm = 64.9 + 41*(composition["G"] + composition["C"] - 16.4) / (composition["A"] + composition["T"] + composition["G"] + composition["C"])
    return (Tm)
def getComposition(sequence):
    composition = {"A" : 0, "G" : 0, "C" : 0, "T" : 0}
    for base in sequence:
        if base == "a":
            composition["A"] = composition["A"] + 1
        elif base == "t":
            composition["T"] = composition["T"] + 1
        elif base == "g":
            composition["G"] = composition["G"] + 1
        elif base == "c":
            composition["C"] = composition["C"] + 1
        else:
            print("error 2")
    return composititon

IndentationError: unexpected indent (<ipython-input-3-5b08cc71c2f6>, line 6)

In [4]:
import primer3

class DNASequence():
    def __init__(self, sequence):
        #Initialize the DNA variable, which has sequence info as well as additional properties (Tm, homodimer & hairpin formation)
        self.sequence = sequence
        self.composition = self.getComposition(sequence)
        if len(sequence) != 0:
            self.gcContent = self.getGCcontent(sequence)
        else:
            self.gcContent = 0
        if len(sequence) < 60:
            self.Tm = primer3.calcTm(sequence)
            self.Homodimer = primer3.calcHomodimer(sequence)
            self.Hairpin = primer3.calcHairpin(sequence)
        else:
            self.Tm = 100000 #set the Tm arbitrarily high
        
    def getComposition(self, sequence):
        composition = {"A" : 0, "G" : 0, "C" : 0, "T" : 0}
        for base in sequence:
            if base == "A":
                composition["A"] = composition["A"] + 1
            elif base == "T":
                composition["T"] = composition["T"] + 1
            elif base == "G":
                composition["G"] = composition["G"] + 1
            elif base == "C":
                composition["C"] = composition["C"] + 1
            else:
                print("error 2")
        return composition
    def getGCcontent(self, sequence):
        GCpercent = (self.composition["G"] + self.composition["C"])/(self.composition["G"] + self.composition["T"] + self.composition["A"] + self.composition["C"])
        return GCpercent
    def reverseComplement(self, sequence):
        #Takes the complement of the sequence and reverses it.
        reverseComplement = ""
        for base in sequence:
            if base == "A":
                reverseComplement = "T" + reverseComplement
            elif base == "T":
                reverseComplement = "A" + reverseComplement
            elif base == "G":
                reverseComplement = "C" + reverseComplement
            elif base == "C":
                reverseComplement = "G" + reverseComplement 
            else:
                print ("error")
        return (reverseComplement)
def reverseComplement(sequence):
    #Takes the complement of the sequence and reverses it.
    reverseComplement = ""
    for base in sequence:
        if base == "A":
            reverseComplement = "T" + reverseComplement 
        elif base == "T":
            reverseComplement = "A" + reverseComplement
        elif base == "G":
            reverseComplement = "C" + reverseComplement
        elif base == "C":
            reverseComplement = "G" + reverseComplement
        else:
            print ("error")
    return (reverseComplement)
def screenPrimers(insert, upstreamBackbone, downstreamBackbone):
    #run the main program to create and screen the forward and reverse primers and then add overhangs.
    forwardPrimer = screenPrimer(insert, "F")
    print(forwardPrimer.sequence, "forward")
    reversePrimer = screenPrimer(insert, "R")
    print(reversePrimer.sequence, "reverse")
    
    primers = addOverhangs(forwardPrimer, reversePrimer, "reverse", upstreamBackbone, downstreamBackbone)
    return (primers[0].sequence, primers[1].sequence)
def screenPrimer(insert, whichPrimer, primerSize=18): #Make primers for an insert to amplify it such that it can be assembled with Gibson
    if primerSize < 40:
        #Make sure you are selecting the right region to prime. 
        #This algorithm creates primers that amplify the WHOLE SEQUENCE minus the stop codon at the end.
        if whichPrimer == "F": 
            primer = DNASequence(insert.sequence[:primerSize])
        elif whichPrimer == "R":
            primer = DNASequence(reverseComplement(insert.sequence[len(insert.sequence) - primerSize: len(insert.sequence)]))
        else:
            print("error")
        #Add various screens below here. You can update the parameters to be more or less stringent depending on how likely you would like your primers to work
        isGood = True
        if primer.Tm < 58.0 or primer.Tm > 59.5:
            isGood = False
        if float(primer.Homodimer.dg) < -10000.0:
            isGood = False
        if float(primer.Hairpin.dg) < -5000.0:
            isGood = False
    
        #return the shortest (18bp ≤) desireable primers
        if isGood == True:
            #screening returned a good primer; return the primer
            return primer
        else:
            #primer failed screening, try again with a larger primer using recursion
            if whichPrimer == "F":
                primer = screenPrimer(insert, "F", primerSize + 1) 
            elif whichPrimer == "R":
                primer = screenPrimer(insert, "R", primerSize + 1)
            return primer
    else:
        #parameters are too stringent; no primers were found that matched the criteria given. Return an empty string
        print ("You cannot make primers with the given parameters")
        return DNASequence("")
def addOverhangs(primer1, primer2, orientation, leftBackbone, rightBackbone):
    #decide how the construct should be oriented in the final vector and add the overhangs so Gibson will add correctly.
    if orientation == "forward":
        primer2.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer2.sequence
        primer1.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer1.sequence
        return primer1, primer2
    elif orientation == "reverse":
        primer1.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer1.sequence
        primer2.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer2.sequence
        return primer1, primer2
    else:
        print ("please enter either a forward or reverse strand orientation")


        
insert = DNASequence('atgGAAAAGACATATAACCCACAAGATATCGAACAGCCGCTTTACGAGCACTGGGAAAAGCTGATTGAACAGCAGGCTGTTATCGCCGCGCTG'.upper())
leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())

rightbackbone = DNASequence('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgcca'.upper())

screenPrimers(insert, leftbackbone , rightbackbone)

ATGGAAAAGACATATAACCCACAAGATATCGAACA forward
CAGCGCGGCGATAACAGCC reverse


('CTAGAAATAATTTTGTTTAACTTTAAGAAGGAGATATACCATGGAAAAGACATATAACCCACAAGATATCGAACA',
 'CTTTGTTAGCAGCCGGATCTCAGTGGTGGTGGTGGTGGTGCAGCGCGGCGATAACAGCC')

In [12]:
reverseComplement('TTTCTGGCGTTGAAGACCGTTTCAATGCGTCGCT'.upper())

'AGCGACGCATTGAAACGGTCTTCAACGCCAGAAA'

In [ ]:
import primer3

primer3.calcTm('atggaaaagacatataacccacaagatatcgaac'.upper())

t = (reverseComplement('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgc'[:40].upper()))

print(t)

leftbackbone = ('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())

def reverse(sequence):
    reverse = ""
    for base in sequence:
        if base == "A":
            reverse = "A" + reverse 
        elif base == "T":
            reverse = "T" + reverse
        elif base == "G":
            reverse = "G" + reverse
        elif base == "C":
            reverse = "C" + reverse
        else:
            print ("error")
    return (reverse)

s = (leftbackbone)

s[len(s) - 40: len(s)]

In [ ]:
def long_substr(data):
    substr = ''
    if len(data) > 1 and len(data[0]) > 0:
        for i in range(len(data[0])):
            for j in range(len(data[0])-i+1):
                if j > len(substr) and is_substr(data[0][i:i+j], data):
                    substr = data[0][i:i+j]
    return substr

def is_substr(find, data):
    if len(data) < 1 and len(find) < 1:
        return False
    for i in range(len(data)):
        if find not in data[i]:
            return False
    return True


print (long_substr(['CCCCATCGATAACCCAGGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGT', 'CTGTAAACCGAGGTTTTGGACTTAGCACCTCTGTTTGGGAAAATTGGGGAGTCGAGTCGGATTTGATCTGATCAAGAGACAGGATGAGGATCGTTTCGCAAAGAGGAGAAAGAGCTCATGATTGAACAAGATGGATTGCACGCAGGTTCTCCGGCCGCTTGGGTGGAGAGGCTATTCGGCTATGACTGGGCACAACAGACAATCGGCTGCTCTGATGCCGCCGTGTTCCGGCTGTCAGCGCAGGGGCGCCCGGTTCTTTTTGTCAAGACCGACCTGTCCGGTGCCCTGAATGAACTTCAGGACGAGGCAGCGCGGCTATCGTGGCTGGCCACGACGGGCGTTCCTTGCGCAGCTGTGCTCGACGTTGTCACTGAAGCGGGAAGGGACTGGCTGCTATTGGGCGAAGTGCCGGGGCAGGATCTCCTGTCATCTCACCTTGCTCCTGCCGAGAAAGTATCCATCATGGCTGATGCAATGCGGCGGCTGCATACGCTTGATCCGGCTACCTGCCCATTCGACCACCAAGCGAAACATCGCATCGAGCGAGCACGTACTCGGATGGAAGCCGGTCTTGTCGATCAGGATGATCTGGACGAAGAGCATCAGGGGCTCGCGCCAGCCGAACTGTTCGCCAGGCTCAAGGCGCGCATGCCCGACGGCGAGGATCTCGTCGTGACCCACGGCGATGCCTGCTTGCCGAATATCATGGTGGAAAATGGCCGCTTTTCTGGATTCATCGACTGTGGCCGGCTGGGTGTGGCGGACCGCTATCAGGACATAGCGTTGGCTACCCGTGATATTGCTGAAGAGCTTGGCGGCGAATGGGCTGACCGCTTCCTCGTGCTTTACGGTATCGCCGCTCCCGATTCGCAGCGCATCGCCTTCTATCGCCTTCTTGACGAGTTCTTCTGAGCGGGACTCTGGGGTTCGAAATGACCGACCAAGCGACGCCCAACCTGCCATCACGAGATTTCGATTCCACCGCCGCCTTCTATGAAAGGTTGGGCTTCGGAATCGTTTTCCGGGACGCCGGCTGGATGATCCTCCAGCGCGGGGATCTCATGCTGGAGTTCTTCGCCCACCCCATCGATAACCCAGGCATCA']))

#findHomology('TCCCTATCAGTGATAGAGAAAAGAGAATCGGATCCAAAAGATCTAAAGAGGAGAAAGGATCTATGTTAATTGGCTATGTCCGTGTGTCCACCAACGATCAAAACACAGATCTTCAAAGAAATGCCCTCGTTTGCGCGGGATGTGAGCAGATCTTTGAAGATAAACTTAGCGGTACCCGTACCGATCGACCGGGTCTTAAACGCGCACTTAAACGGCTTCAGAAGGGGGATACCTTGGTCGTATGGAAACTGGACCGGCTGGGTAGATCGATGAAACATCTGATTAGTTTAGTTGGAGAACTGCGAGAACGCGGCATCAATTTTCGCTCCTTAACCGATAGCATCGATACCTCCAGTCCAATGGGGAGGTTTTTCTTCTATGTTATGGGCGCGCTGGCCGAAATGGAACGTGAATTGATTATCGAGCGGACAATGGCTGGGTTAGCGGCAGCACGAAATAAAGGACGGCGCTTCGGTCGTCCACCCAAGGGCGGCTCCGGTGGCTCGGGAGGTTCCGGCGGTAGCGGGGGCAGCGGCGGCAGTGGGGGCTCAGGAGGTTCTGATAAAAAATATTCTATAGGACTGGCTATTGGTACCAATTCAGTGGGTTGGGCGGTAATTACCGACGAATATAAAGTGCCGTCTAAAAAATTTAAGGTCTTGGGGAATACAGACCGTCACAGTATAAAAAAAAATCTGATCGGTGCGCTGCTGTTTGACTCTGGAGAAACTGCCGAAGCAACTCGTTTAAAACGAACCGCACGCCGTCGCTATACAAGGCGGAAAAATCGTATATGCTATTTACAAGAAATTTTTTCCAATGAAATGGCAAAAGTCGATGACAGCTTCTTTCATCGCTTAGAAGAGTCCTTTTTAGTTGAAGAAGATAAAAAACACGAACGCCATCCAATTTTTGGGAATATAGTTGATGAAGTCGCCTATCACGAAAAATATCCGACTATTTACCACCTGCGTAAGAAACTTGTGGATTCGACGGATAAAGCAGATCTGAGGCTGATTTATCTCGCCCTCGCACATATGATTAAATTCAGAGGTCATTTCCTGATAGAAGGCGATCTGAATCCCGACAACTCTGATGTTGATAAGTTGTTTATACAGCTGGTGCAAACCTATAATCAACTATTCGAAGAGAATCCTATAAACGCCAGCGGAGTGGACGCTAAAGCGATACTCAGCGCGCGCTTATCTAAAAGCCGGCGGCTGGAAAATCTGATCGCACAATTGCCAGGTGAAAAAAAAAATGGTTTATTTGGCAATCTGATAGCGCTTTCACTGGGGCTAACACCCAATTTTAAATCGAATTTTGATTTAGCCGAAGATGCTAAGCTGCAGCTGAGTAAAGATACCTATGATGACGATCTTGATAACCTGCTGGCACAAATTGGAGATCAATACGCAGATCTGTTTTTGGCCGCCAAGAATCTGTCGGATGCAATTCTGCTGAGCGATATTTTGAGGGTGAATACTGAAATCACTAAGGCACCGCTCTCTGCAAGTATGATCAAGCGTTATGATGAGCATCACCAGGACTTAACCCTGTTAAAAGCTCTGGTCCGCCAGCAGCTCCCGGAAAAATACAAGGAAATATTCTTCGATCAATCTAAGAATGGGTATGCCGGCTATATCGATGGCGGAGCGTCTCAGGAAGAATTTTATAAATTCATAAAACCTATCCTTGAAAAAATGGATGGAACGGAAGAGCTGCTGGTTAAACTGAATAGAGAGGATCTGTTACGCAAGCAGAGGACATTCGATAACGGTAGTATCCCACATCAAATACATCTGGGTGAACTTCACGCGATTCTTCGCCGCCAGGAAGACTTTTATCCCTTTCTAAAAGACAACAGGGAGAAAATTGAAAAAATCCTGACGTTTCGTATTCCTTATTATGTGGGGCCGCTGGCTCGTGGAAACAGTCGGTTCGCATGGATGACGCGCAAGTCCGAAGAGACGATAACGCCGTGGAATTTCGAAGAAGTGGTCGACAAAGGCGCAAGTGCACAGTCATTCATCGAACGTATGACTAACTTCGATAAAAATTTACCAAACGAGAAAGTTCTGCCTAAGCATAGCCTTCTCTATGAATACTTTACAGTCTATAACGAGTTAACTAAAGTTAAATATGTGACTGAAGGTATGCGTAAACCGGCGTTTCTGAGCGGCGAACAAAAAAAGGCAATTGTCGATCTGTTATTTAAAACAAATCGTAAAGTTACGGTCAAGCAATTGAAAGAGGACTACTTCAAAAAAATTGAGTGTTTCGACAGCGTCGAAATTTCTGGCGTTGAAGACCGTTTCAAT', 'CGTTGAAGACCGTTTCAATGCGTCGCTGGGAACTTACCACGATCTACTCAAAATCATAAAAGATAAAGACTTTCTGGACAATGAAGAGAACGAAGACATACTTGAAGACATCGTTCTGACCCTTACGCTATTTGAAGATAGGGAGATGATCGAAGAGCGCTTGAAGACCTATGCGCACCTATTCGATGATAAAGTTATGAAACAGCTCAAGCGGCGCCGTTACACGGGGTGGGGGCGTCTGTCGAGGAAGTTAATCAATGGAATTCGTGATAAGCAGTCAGGCAAGACGATCCTGGACTTTCTGAAATCAGATGGGTTTGCAAATCGTAACTTCATGCAGCTGATCCATGATGATTCGTTAACTTTTAAAGAAGATATTCAAAAAGCCCAAGTATCTGGTCAGGGTGACTCTCTTCACGAACATATCGCGAATTTGGCAGGTTCACCAGCTATTAAAAAAGGTATCTTGCAGACTGTGAAAGTTGTCGATGAACTTGTTAAAGTTATGGGTCGTCACAAGCCAGAGAATATCGTAATTGAGATGGCGCGTGAAAATCAAACAACCCAAAAAGGCCAGAAAAACTCGAGGGAACGCATGAAACGTATTGAAGAAGGTATTAAGGAGTTGGGTTCCCAGATACTTAAAGAACACCCGGTCGAAAATACGCAACTGCAAAATGAAAAATTATATCTATATTATCTTCAGAACGGGCGCGATATGTATGTGGACCAAGAGCTGGATATCAATCGGCTTTCGGATTATGACGTCGATGCAATCGTTCCGCAAAGTTTCCTCAAGGATGACAGCATCGATAACAAAGTGTTAACCCGTAGCGATAAAAACCGTGGCAAAAGTGATAACGTGCCATCGGAAGAGGTAGTGAAAAAGATGAAAAATTATTGGCGCCAGCTGCTTAATGCTAAGCTGATTACGCAACGGAAATTTGATAATCTGACTAAAGCTGAGCGGGGCGGGCTGTCGGAACTCGATAAAGCTGGGTTCATAAAAAGACAATTGGTCGAGACGCGCCAGATTACCAAACACGTAGCCCAAATTCTGGATTCACGGATGAATACGAAATACGACGAGAATGATAAACTGATCCGGGAAGTGAAAGTGATCACTCTTAAATCTAAACTGGTATCAGACTTTAGAAAAGATTTCCAATTCTATAAGGTTCGTGAAATCAACAATTATCATCACGCTCATGACGCCTATTTAAATGCGGTGGTAGGCACAGCGTTAATCAAGAAATATCCGAAACTTGAGTCAGAGTTTGTGTATGGAGATTATAAAGTTTATGATGTTAGAAAAATGATTGCTAAATCGGAGCAAGAAATTGGTAAGGCTACAGCGAAATATTTTTTTTACTCCAATATTATGAATTTCTTTAAGACTGAAATTACCTTGGCTAATGGTGAAATCCGCAAACGGCCGCTGATCGAAACAAATGGTGAAACTGGCGAGATTGTTTGGGATAAGGGCCGCGATTTCGCAACGGTCCGTAAGGTGTTGAGTATGCCTCAGGTGAATATTGTAAAGAAAACGGAAGTACAGACTGGCGGGTTTAGCAAAGAATCAATTTTGCCGAAACGTAATTCTGATAAACTGATCGCTCGTAAGAAAGATTGGGATCCCAAGAAGTATGGTGGCTTTGATAGTCCGACCGTAGCGTATAGTGTTCTGGTGGTAGCTAAGGTAGAAAAAGGCAAATCAAAGAAACTGAAGTCCGTGAAGGAACTACTGGGTATCACGATCATGGAACGATCAAGTTTTGAAAAAAATCCGATCGACTTCCTTGAAGCGAAAGGGTACAAAGAGGTAAAGAAGGACCTAATTATAAAACTGCCCAAATATAGTCTGTTTGAACTGGAAAATGGAAGGAAACGAATGTTGGCAAGCGCTGGTGAGCTGCAGAAAGGCAATGAATTAGCCCTGCCCTCGAAATATGTAAATTTCTTATACTTGGCCAGTCATTATGAAAAGCTGAAAGGATCACCTGAAGATAATGAACAAAAACAGCTCTTCGTCGAGCAACACAAGCATTACTTAGATGAAATTATAGAACAAATTTCTGAATTCTCAAAACGTGTTATATTAGCGGACGCGAATCTGGATAAGGTCTTGAGCGCATATAACAAGCATCGCGATAAACCGATACGGGAACAAGCGGAAAATATTATACATCTGTTCACGCTGACCAATTTAGGCGCGCCTGCTGCGTTTAAATATTTTGATACAACCATAGATCGTAAACGTTACACCAGCACAAAGGAAGTCCTGGATGCCACACTGATCCATCAGAGCATAACAGGACTGTACGAGACCCGTATCGATCTGAGTCAGCTTGGAGGTGATGGTGGTTCGGATTACAAGGATGACGATGATAAAGGTGGAAGTCCGAAAAAAAAAAGGAAAGTTTGATAACTCGAGTAAGGATCAGTAAGGATCTCCAGGCATC')

In [5]:
import primer3
import pandas
import math
import requests
#import pandas as pd
import numpy as np

sequences = []
breakdown = []

seqPrimers = []
seqPrimerNames = []

data = pandas.read_excel("/Users/aleclourenco/Desktop/REU-2017-Endy-Lab/PURE Datasheet.xlsx", sheetname="List of PURE Components", skiprows=[0])

class DNASequence():
    def __init__(self, sequence):
        #Initialize the DNA variable, which has sequence info as well as additional properties (Tm, homodimer & hairpin formation)
        self.sequence = sequence
        self.composition = self.getComposition(sequence)
        if len(sequence) != 0:
            self.gcContent = self.getGCcontent(sequence)
        else:
            self.gcContent = 0
        if len(sequence) < 60:
            self.Tm = primer3.calcTm(sequence)
            self.Homodimer = primer3.calcHomodimer(sequence)
            self.Hairpin = primer3.calcHairpin(sequence)
        else:
            self.Tm = 100000 #set the Tm arbitrarily high
        
    def getComposition(self, sequence):
        composition = {"A" : 0, "G" : 0, "C" : 0, "T" : 0}
        for base in sequence:
            if base == "A":
                composition["A"] = composition["A"] + 1
            elif base == "T":
                composition["T"] = composition["T"] + 1
            elif base == "G":
                composition["G"] = composition["G"] + 1
            elif base == "C":
                composition["C"] = composition["C"] + 1
            else:
                print("error 2")
        return composition
    def getGCcontent(self, sequence):
        GCpercent = (self.composition["G"] + self.composition["C"])/(self.composition["G"] + self.composition["T"] + self.composition["A"] + self.composition["C"])
        return GCpercent
    def reverseComplement(self, sequence):
        #Takes the complement of the sequence and reverses it.
        reverseComplement = ""
        for base in sequence:
            if base == "A":
                reverseComplement = "T" + reverseComplement
            elif base == "T":
                reverseComplement = "A" + reverseComplement
            elif base == "G":
                reverseComplement = "C" + reverseComplement
            elif base == "C":
                reverseComplement = "G" + reverseComplement 
            else:
                print ("error")
        return (reverseComplement)
def reverseComplement(sequence):
    #Takes the complement of the sequence and reverses it.
    reverseComplement = ""
    for base in sequence:
        if base == "A":
            reverseComplement = "T" + reverseComplement 
        elif base == "T":
            reverseComplement = "A" + reverseComplement
        elif base == "G":
            reverseComplement = "C" + reverseComplement
        elif base == "C":
            reverseComplement = "G" + reverseComplement
        else:
            print ("error")
    return (reverseComplement)
def screenPrimers(insert, upstreamBackbone, downstreamBackbone):
    #run the main program to create and screen the forward and reverse primers and then add overhangs.
    forwardPrimer = screenPrimer(insert, "F")
    #print(forwardPrimer.sequence, "forward")
    reversePrimer = screenPrimer(insert, "R")
    #print(reversePrimer.sequence, "reverse")
    if len(forwardPrimer.sequence) and len(reversePrimer.sequence) != 0:
        primers = addOverhangs(forwardPrimer, reversePrimer, "reverse", upstreamBackbone, downstreamBackbone)
        return (primers[0].sequence, primers[1].sequence)
    else:
        return ("You cannot make primers with the given parameters", "You cannot make primers with the given parameters")
    
def screenPrimer(insert, whichPrimer, primerSize=18): #Make primers for an insert to amplify it such that it can be assembled with Gibson
    if primerSize < 40:
        #Make sure you are selecting the right region to prime. 
        #This algorithm creates primers that amplify the WHOLE SEQUENCE minus the stop codon at the end.
        noncanonicalStartCodon = False
        if whichPrimer == "F": 
            primer = DNASequence(insert.sequence[:primerSize])
            if (primer.sequence[:3].lower() != "atg"):
                primer = DNASequence(insert.sequence[3:primerSize])
                noncanonicalStartCodon = True
        elif whichPrimer == "R":
            primer = DNASequence(reverseComplement(insert.sequence[len(insert.sequence) - primerSize: len(insert.sequence)]))
        else:
            print("error")
        #Add various screens below here. You can update the parameters to be more or less stringent depending on how likely you would like your primers to work
        isGood = True
        if primer.Tm < 58.0 or primer.Tm > 59.5:
            isGood = False
        if float(primer.Homodimer.dg) < -10000.0:
            isGood = False
        if float(primer.Hairpin.dg) < -5000.0:
            isGood = False
    
        #return the shortest (18bp ≤) desireable primers
        if isGood == True:
            #screening returned a good primer; return the primer
            if noncanonicalStartCodon == True:
                primer = DNASequence("atg".upper() + primer.sequence)
            return primer
        else:
            #primer failed screening, try again with a larger primer using recursion
            if whichPrimer == "F":
                primer = screenPrimer(insert, "F", primerSize + 1) 
            elif whichPrimer == "R":
                primer = screenPrimer(insert, "R", primerSize + 1)
            return primer
    else:
        #parameters are too stringent; no primers were found that matched the criteria given. Return an empty string
        return DNASequence("")
def addOverhangs(primer1, primer2, orientation, leftBackbone, rightBackbone):
    #decide how the construct should be oriented in the final vector and add the overhangs so Gibson will add correctly.
    if orientation == "forward":
        primer2.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer2.sequence
        primer1.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer1.sequence
        return primer1, primer2
    elif orientation == "reverse":
        primer1.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer1.sequence
        primer2.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer2.sequence
        return primer1, primer2
    else:
        print ("please enter either a forward or reverse strand orientation")
def formatSeq(oldSeq):
    if not type(oldSeq) is str:
        oldSeq = ""
    new = oldSeq.replace("\n", "")
    for number in range(0,10):
        temp = new.replace(str(number), "")
        new = temp
    oldSeq = new
    new = oldSeq.replace(" ", "")
    return new.upper()

def appendPrimers(data, insert, tag, primerList, nameList):
    name = str(data.ix[number,"Protein Name"]).replace(" ", "") 
    nameFOR = name + " FOR " + tag + "-tagged"
    nameREV = name + " REV " + tag + "-tagged"
    primerList.append(screenPrimers(insert, leftbackbone , rightbackbone)[0])
    nameList.append(nameFOR)
    primerList.append(screenPrimers(insert, leftbackbone , rightbackbone)[1])
    nameList.append(nameREV)
def unpackPrimers(nameList, seqList, finNameList, finSeqList):
    for names in nameList:
        finNameList.append(names)
    for primers in seqList:
        finSeqList.append(primers)
def createVector(insert, tag, name):
    seqPrimerList = []
    seqPrimerNameList = []
    insert = insert.sequence
    annotations = []
    pBR322_origin = "cgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatct".upper()
    kanR2 = "atgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaa".upper()
    rop = "gtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctga".upper()
    t7Terminator = "atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcag".upper()
    tet300 = "gccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagag".upper()
    pGEX_3_primer = "cctctgacacatgcagctcccgg".upper()
    thrombin = thrombin = "gctgccgcgcggcaccag".upper()
    t7promoter = "cctatagtgagtcgtatta".upper()
    lacO = "ggggaattgttatccgctcacaattccc".upper()
    hisTag = "gtgatgatgatgatgatg".upper()
    seqRev = "gcctatatcgccgacatcacc".upper()
    seqFor = "tccggatatagttcctcctttcagc".upper()
    rbs = "ctccttcttaaagttaa".upper()
    hisPlusStop = "tcagtggtggtggtggtggtg".upper()
    lacI = "gtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtga".upper()
    tet576 = "cccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatg".upper()
    
    
    if tag =="C":
        leftBackbone = "cgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgccaatccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg".upper()
        rightBackbone = "ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcaggg".upper()
        sequence = leftBackbone + reverseComplement(insert[:len(insert) - 3]) + rightBackbone
        endLen = len(leftBackbone) + len(insert) - 3
        addAnnotation(annotations, reverseComplement(insert[:len(insert) - 3]), sequence, name, 'gene', "#0000ff")
        
    if tag =="N":
        leftBackbone = "cgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgccaatccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatc".upper()
        rightBackbone = "atggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcg".upper()
        sequence = leftBackbone + reverseComplement(insert) + rightBackbone
        endLen = len(leftBackbone) + len(insert)
        addAnnotation(annotations, reverseComplement(insert), sequence, name, 'gene', "#0000ff")
        
    offName = str(name) + " " + tag + "-tagged"
    
    beta = findSeqPrimers(DNASequence(seqFor), DNASequence(reverseComplement(seqRev)), DNASequence(sequence))
    count = 0
    for alpha in beta:
        count += 1
        nom = "seqFOR" + str(count) + " " + offName
        addAnnotation(annotations, alpha, sequence, nom, "primer", "#f2c22e")
        seqPrimerNameList.append(nom)
        seqPrimerList.append(alpha)
    
    addAnnotation(annotations, pBR322_origin, sequence, "pBR322 Origin", "origin", "#9faa9a")
    addAnnotation(annotations, kanR2, sequence, "Kan R2", "gene", "#359797")
    addAnnotation(annotations, t7Terminator, sequence, "T7 Terminator", "terminator", "#d24b19")
    addAnnotation(annotations, tet300, sequence, "tet (300-563)", "gene", "#ffe2b9")
    addAnnotation(annotations, pGEX_3_primer, sequence, "pGEX_3_primer", "primer", "#5b637e")
    addAnnotation(annotations, thrombin, sequence, "thrombin", "tag", "#5f631d")
    addAnnotation(annotations, t7promoter, sequence, "T7 Promoter", "promoter", "#c8498b")
    addAnnotation(annotations, lacO, sequence, "lacO", "regulatory", "#fffbaa")
    addAnnotation(annotations, lacO, sequence, "lacI", "regulatory", "#c3dff3")
    addAnnotation(annotations, hisTag, sequence, "His Tag", "tag", "#f9c5fe")
    addAnnotation(annotations, seqFor, sequence, "pET28+ Forward Primer", "primer", "#fffafa")
    addAnnotation(annotations, seqRev, sequence, "pET28+ Reverse Primer", "primer", "#fffafa")
    addAnnotation(annotations, rbs, sequence, "T7 Ribosome Binding Site", "feature", "#e12222")
    addAnnotation(annotations, hisPlusStop, sequence, "His Tag and Stop Codon", "feature", "#ddb34e")
    addAnnotation(annotations, tet576, sequence, "tet(576-851)", "feature", "#ba9231")
    addAnnotation(annotations, rop, sequence, "ROP", "feature", "#1ebdac")
    
    
    
    data = {
      "aliases": [],
      "annotations": annotations,
      "bases": sequence,
      "circular": True,
      "description": "program generated",
      "folder": "lib_iP53B30G",
      "name": offName,
      "tags": []
    }

    key = 'sk_gnGGnlq0laAcbBcI9yVEIqpE7opJQ'
    resp = requests.post('https://api.benchling.com/v1/sequences/', json=data, auth=(key, ''))
    print(resp)
    return (seqPrimerNameList, seqPrimerList)

def findSequence(sequence, plasmid):
    if sequence in plasmid:
        a = plasmid.find(sequence)
        #print(a, a + len(sequence))
    else:
        return (-1, -1)
    return (a, a + len(sequence))
def addAnnotation(annotations, sequence, plasmid, name, annotationType, color):
    annotation = {
          "color": color,
          "end": findSequence(sequence, plasmid)[1],
          "name": name,
          "start": findSequence(sequence, plasmid)[0],
          "strand": 0,
          "type": annotationType
    }
    if findSequence(sequence, plasmid) != (-1, -1):
        annotations.append(annotation)
    return annotations

primerList = []
nameList = []

for number in range(0,49):
    tag = ""
    if (data.ix[number, "His Tagged Where?"]) == "C":
        tag = "C"
        #Correct the backbone so that the primers generated add in the correct area of the backbone
        leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())
        rightbackbone = DNASequence('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgcca'.upper())
        insert = DNASequence(formatSeq(data.ix[number,"Nucleotide Sequence"]))
        name = data.ix[number,"Protein Name"]
        a = createVector(insert, tag, name)
        unpackPrimers(a[0], a[1], seqPrimerNames, seqPrimers)
        insert = DNASequence(insert.sequence[:len(insert.sequence) - 3])
        if len(insert.sequence) != 0:
            appendPrimers(data, insert, tag, primerList, nameList)
    elif (data.ix[number, "His Tagged Where?"]) == "N": 
        tag = "N"
        #Correct the backbone so that the primers generated add in the correct area of the backbone
        leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatc'.upper())
        rightbackbone = DNASequence('atggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccag'.upper())
        insert = DNASequence(formatSeq(data.ix[number,"Nucleotide Sequence"]))
        name = data.ix[number,"Protein Name"]
        a = createVector(insert, tag, name)
        unpackPrimers(a[0], a[1], seqPrimerNames, seqPrimers)
        if len(insert.sequence) != 0:
            appendPrimers(data, insert, tag, primerList, nameList)
    else: #We don't know whether unspecified proteins are N or C His-tagged, so we assume they could be either.
        leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())
        rightbackbone = DNASequence('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgcca'.upper())
        insert = DNASequence(formatSeq(data.ix[number,"Nucleotide Sequence"]))
        if len(insert.sequence) != 0:
            tag = "C"
            insert = DNASequence(insert.sequence[:len(insert.sequence) - 3])
            name = data.ix[number,"Protein Name"]
            a = createVector(insert, tag, name)
            appendPrimers(data, insert, tag, primerList, nameList)
            unpackPrimers(a[0], a[1], seqPrimerNames, seqPrimers)
        leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatc'.upper())
        rightbackbone = DNASequence('atggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccag'.upper())
        insert = DNASequence(formatSeq(data.ix[number,"Nucleotide Sequence"]))
        if len(insert.sequence) != 0:
            tag = "N"
            name = data.ix[number,"Protein Name"]
            a = createVector(insert, tag, name)
            appendPrimers(data, insert, tag, primerList, nameList)
            unpackPrimers(a[0], a[1], seqPrimerNames, seqPrimers)
        
dataframe = pandas.DataFrame(primerList, nameList)
df = pandas.DataFrame(seqPrimers, seqPrimerNames)

writer = pandas.ExcelWriter('primerList.xlsx', engine='openpyxl')
writer2 = pandas.ExcelWriter('seqPrimerList.xlsx', engine='openpyxl')

dataframe.to_excel(writer)
df.to_excel(writer2)

writer.save()
writer2.save()

<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [400]>
<Response [400]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>


In [38]:
a = screenPrimer(DNASequence('ggaagaagctggcgctgaagttgaagttaaataagccaacccttccggttgcagcctga'.upper()), 'F')
print(a.sequence)

b = screenPrimer(DNASequence('ggaagaagctggcgctgaagttgaagttaaataagccaacccttccggttgcagcctga'.upper()), 'R')
print(b.sequence)

ATGAGAAGCTGGCGCTGAAGTTGAAGT
TCAGGCTGCAACCGGAAGGG


In [49]:
leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatc'.upper())
rightbackbone = DNASequence('atggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccag'.upper())

a = addOverhangs(DNASequence("atgaacacgattaacatcgctaagaacga".upper()), DNASequence("CTATGCAAACGCGAAGTCCGACTCTAAGATG"), 'reverse', leftbackbone, rightbackbone)

print(a[1].sequence)

CCAACTCAGCTTCCTTTCGGGCTTTGTTAGCAGCCGGATCCTATGCAAACGCGAAGTCCGACTCTAAGATG


In [12]:
import primer3
import pandas
import math
sequences = []
breakdown = []

seqPrimers = []
seqPrimerNames = []

data = pandas.read_excel("/Users/aleclourenco/Desktop/REU-2017-Endy-Lab/PURE Datasheet.xlsx", sheetname='Alec\'s Chaperones', skiprows=[0])

for number in range (0, 7):
    print(formatSeq(data.ix[number,"DNA Sequence"]))
    print ("\n")
    print(number)
class DNASequence():
    def __init__(self, sequence):
        #Initialize the DNA variable, which has sequence info as well as additional properties (Tm, homodimer & hairpin formation)
        self.sequence = sequence
        self.composition = self.getComposition(sequence)
        if len(sequence) != 0:
            self.gcContent = self.getGCcontent(sequence)
        else:
            self.gcContent = 0
        if len(sequence) < 60:
            self.Tm = primer3.calcTm(sequence)
            self.Homodimer = primer3.calcHomodimer(sequence)
            self.Hairpin = primer3.calcHairpin(sequence)
        else:
            self.Tm = 100000 #set the Tm arbitrarily high
        
    def getComposition(self, sequence):
        composition = {"A" : 0, "G" : 0, "C" : 0, "T" : 0}
        for base in sequence:
            if base == "A":
                composition["A"] = composition["A"] + 1
            elif base == "T":
                composition["T"] = composition["T"] + 1
            elif base == "G":
                composition["G"] = composition["G"] + 1
            elif base == "C":
                composition["C"] = composition["C"] + 1
            else:
                print("error 2")
        return composition
    def getGCcontent(self, sequence):
        GCpercent = (self.composition["G"] + self.composition["C"])/(self.composition["G"] + self.composition["T"] + self.composition["A"] + self.composition["C"])
        return GCpercent
    def reverseComplement(self, sequence):
        #Takes the complement of the sequence and reverses it.
        reverseComplement = ""
        for base in sequence:
            if base == "A":
                reverseComplement = "T" + reverseComplement
            elif base == "T":
                reverseComplement = "A" + reverseComplement
            elif base == "G":
                reverseComplement = "C" + reverseComplement
            elif base == "C":
                reverseComplement = "G" + reverseComplement 
            else:
                print ("error")
        return (reverseComplement)
def reverseComplement(sequence):
    #Takes the complement of the sequence and reverses it.
    reverseComplement = ""
    for base in sequence:
        if base == "A":
            reverseComplement = "T" + reverseComplement 
        elif base == "T":
            reverseComplement = "A" + reverseComplement
        elif base == "G":
            reverseComplement = "C" + reverseComplement
        elif base == "C":
            reverseComplement = "G" + reverseComplement
        else:
            print ("error")
    return (reverseComplement)
def screenPrimers(insert, upstreamBackbone, downstreamBackbone):
    #run the main program to create and screen the forward and reverse primers and then add overhangs.
    forwardPrimer = screenPrimer(insert, "F")
    #print(forwardPrimer.sequence, "forward")
    reversePrimer = screenPrimer(insert, "R")
    #print(reversePrimer.sequence, "reverse")
    if len(forwardPrimer.sequence) and len(reversePrimer.sequence) != 0:
        primers = addOverhangs(forwardPrimer, reversePrimer, "reverse", upstreamBackbone, downstreamBackbone)
        return (primers[0].sequence, primers[1].sequence)
    else:
        return ("You cannot make primers with the given parameters", "You cannot make primers with the given parameters")
    
def screenPrimer(insert, whichPrimer, primerSize=18): #Make primers for an insert to amplify it such that it can be assembled with Gibson
    if primerSize < 40:
        #Make sure you are selecting the right region to prime. 
        #This algorithm creates primers that amplify the WHOLE SEQUENCE minus the stop codon at the end.
        noncanonicalStartCodon = False
        if whichPrimer == "F": 
            primer = DNASequence(insert.sequence[:primerSize])
            if (primer.sequence[:3].lower() != "atg"):
                primer = DNASequence(insert.sequence[3:primerSize])
                noncanonicalStartCodon = True
        elif whichPrimer == "R":
            primer = DNASequence(reverseComplement(insert.sequence[len(insert.sequence) - primerSize: len(insert.sequence)]))
        else:
            print("error")
        #Add various screens below here. You can update the parameters to be more or less stringent depending on how likely you would like your primers to work
        isGood = True
        if primer.Tm < 58.0 or primer.Tm > 59.5:
            isGood = False
        if float(primer.Homodimer.dg) < -10000.0:
            isGood = False
        if float(primer.Hairpin.dg) < -5000.0:
            isGood = False
    
        #return the shortest (18bp ≤) desireable primers
        if isGood == True:
            #screening returned a good primer; return the primer
            if noncanonicalStartCodon == True:
                primer = DNASequence("atg".upper() + primer.sequence)
            return primer
        else:
            #primer failed screening, try again with a larger primer using recursion
            if whichPrimer == "F":
                primer = screenPrimer(insert, "F", primerSize + 1) 
            elif whichPrimer == "R":
                primer = screenPrimer(insert, "R", primerSize + 1)
            return primer
    else:
        #parameters are too stringent; no primers were found that matched the criteria given. Return an empty string
        return DNASequence("")
def addOverhangs(primer1, primer2, orientation, leftBackbone, rightBackbone):
    #decide how the construct should be oriented in the final vector and add the overhangs so Gibson will add correctly.
    if orientation == "forward":
        primer2.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer2.sequence
        primer1.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer1.sequence
        return primer1, primer2
    elif orientation == "reverse":
        primer1.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer1.sequence
        primer2.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer2.sequence
        return primer1, primer2
    else:
        print ("please enter either a forward or reverse strand orientation")
def formatSeq(oldSeq):
    if not type(oldSeq) is str:
        oldSeq = ""
    new = oldSeq.replace("\n", "")
    for number in range(0,10):
        temp = new.replace(str(number), "")
        new = temp
    oldSeq = new
    new = oldSeq.replace(" ", "")
    return new.upper()

def appendPrimers(data, insert, tag, primerList, nameList):
    name = str(data.ix[number,"Chaperone Name"]).replace(" ", "") 
    nameFOR = name + " FOR " + tag + "-tagged"
    nameREV = name + " REV " + tag + "-tagged"
    primerList.append(screenPrimers(insert, leftbackbone , rightbackbone)[0])
    nameList.append(nameFOR)
    primerList.append(screenPrimers(insert, leftbackbone , rightbackbone)[1])
    nameList.append(nameREV)
def unpackPrimers(nameList, seqList, finNameList, finSeqList):
    for names in nameList:
        finNameList.append(names)
    for primers in seqList:
        finSeqList.append(primers)


primerList = []
nameList = []

for number in range(0,7):
    tag = ""
    #C tag them all for now
    leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())
    rightbackbone = DNASequence('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgcca'.upper())
    
    insert = DNASequence(formatSeq(data.ix[number,"DNA Sequence"]))
    insert = DNASequence(insert.sequence[:len(insert.sequence) - 3])
    if len(insert.sequence) != 0:
        tag = "C"
        name = str(data.ix[number,"Chaperone Name"]).replace(" ", "")
        a = createVector(insert, tag, name)
        appendPrimers(data, insert, tag, primerList, nameList)
        if a != None:
            unpackPrimers(a[0], a[1], seqPrimerNames, seqPrimers)
        
dataframe = pandas.DataFrame(primerList, nameList)
df = pandas.DataFrame(seqPrimers, seqPrimerNames)

writer = pandas.ExcelWriter('primerList3.xlsx', engine='openpyxl')
writer2 = pandas.ExcelWriter('seqPrimerListChap.xlsx', engine='openpyxl')

dataframe.to_excel(writer)
df.to_excel(writer2)

writer.save()
writer2.save()

ATGGCTAAGCAAGATTATTACGAGATTTTAGGCGTTTCCAAAACAGCGGAAGAGCGTGAAATCAGAAAGGCCTACAAACGCCTGGCCATGAAATACCACCCGGACCGTAACCAGGGTGACAAAGAGGCCGAGGCGAAATTTAAAGAGATCAAGGAAGCTTATGAAGTTCTGACCGACTCGCAAAAACGTGCGGCATACGATCAGTATGGTCATGCTGCGTTTGAGCAAGGTGGCATGGGCGGCGGCGGTTTTGGCGGCGGCGCAGACTTCAGCGATATTTTTGGTGACGTTTTCGGCGATATTTTTGGCGGCGGACGTGGTCGTCAACGTGCGGCGCGCGGTGCTGATTTACGCTATAACATGGAGCTCACCCTCGAAGAAGCTGTACGTGGCGTGACCAAAGAGATCCGCATTCCGACTCTGGAAGAGTGTGACGTTTGCCACGGTAGCGGTGCAAAACCAGGTACACAGCCGCAGACTTGTCCGACCTGTCATGGTTCTGGTCAGGTGCAGATGCGCCAGGGATTCTTCGCTGTACAGCAGACCTGTCCACACTGTCAGGGCCGCGGTACGCTGATCAAAGATCCGTGCAACAAATGTCATGGTCATGGTCGTGTTGAGCGCAGCAAAACGCTGTCCGTTAAAATCCCGGCAGGGGTGGACACTGGAGACCGCATCCGTCTTGCGGGCGAAGGTGAAGCGGGCGAGCATGGCGCACCGGCAGGCGATCTGTACGTTCAGGTTCAGGTTAAACAGCACCCGATTTTCGAGCGTGAAGGCAACAACCTGTATTGCGAAGTCCCGATCAACTTCGCTATGGCGGCGCTGGGTGGCGAAATCGAAGTACCGACCCTTGATGGTCGCGTCAAACTGAAAGTGCCTGGCGAAACCCAGACCGGTAAGCTATTCCGTATGCGCGGTAAAGGCGTCAAGTCTGTCCGCGGTGGCGCACAGGGTGATTTGCTGTGCCGCGTTGTCGTCGAAACACCGGTAGGCCTGA

In [ ]:
import datetime as dt
import pandas as pd
from openpyxl import load_workbook

def add_frame_to_workbook(filename, tabname, dataframe, timestamp):
    """
    Save a dataframe to a workbook tab with the filename and tabname
    coded to timestamp

    :param filename: filename to create, can use strptime formatting
    :param tabname: tabname to create, can use strptime formatting
    :param dataframe: dataframe to save to workbook
    :param timestamp: timestamp associated with dataframe
    :return: None
    """
    filename = timestamp.strftime(filename)
    sheet_name = timestamp.strftime(tabname)

    # create a writer for this month and year
    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # copy existing sheets
        writer.sheets = dict(
            (ws.title, ws) for ws in writer.book.worksheets)
    except IOError:
        # file does not exist yet, we will create it
        pass

    # write out the new sheet
    dataframe.to_excel(writer, sheet_name=sheet_name)

    # save the workbook
    writer.save()

data = [x.strip().split() for x in """
            Primer Name  Sequence
                    FOR  ATGTGCCATGA...
                    REV  TGCCTAGTCA...
""".split('\n')[1:-1]]
df = pd.DataFrame(data=data[1:], columns=data[0])

name_template = './sample-%m-%y.xlsx'
tab_template = '%d_%H_%M'
now = dt.datetime.now()
in_an_hour = now + dt.timedelta(hours=1)
add_frame_to_workbook(name_template, tab_template, df, now)
add_frame_to_workbook(name_template, tab_template, df, in_an_hour)

In [ ]:
import pandas as pd
import numpy as np


df = pd.DataFrame({'name' : pd.Series(np.random.randn(3)),'Sequence' : pd.Series(np.random.randn(4))})
print(df)

writer = pd.ExcelWriter('test2.xlsx', engine='openpyxl')

df.to_excel(writer)

writer.save()

In [4]:
import primer3
import pandas
import math
sequences = []
breakdown = []
data = pandas.read_excel("/Users/aleclourenco/Desktop/REU-2017-Endy-Lab/PURE Datasheet.xlsx", sheetname="List of Ribosome Protein Compon")

class DNASequence():
    def __init__(self, sequence):
        #Initialize the DNA variable, which has sequence info as well as additional properties (Tm, homodimer & hairpin formation)
        self.sequence = sequence
        self.composition = self.getComposition(sequence)
        if len(sequence) != 0:
            self.gcContent = self.getGCcontent(sequence)
        else:
            self.gcContent = 0
        if len(sequence) < 60:
            self.Tm = primer3.calcTm(sequence)
            self.Homodimer = primer3.calcHomodimer(sequence)
            self.Hairpin = primer3.calcHairpin(sequence)
        else:
            self.Tm = 100000 #set the Tm arbitrarily high
        
    def getComposition(self, sequence):
        composition = {"A" : 0, "G" : 0, "C" : 0, "T" : 0}
        for base in sequence:
            if base == "A":
                composition["A"] = composition["A"] + 1
            elif base == "T":
                composition["T"] = composition["T"] + 1
            elif base == "G":
                composition["G"] = composition["G"] + 1
            elif base == "C":
                composition["C"] = composition["C"] + 1
            else:
                print("error 2")
        return composition
    def getGCcontent(self, sequence):
        GCpercent = (self.composition["G"] + self.composition["C"])/(self.composition["G"] + self.composition["T"] + self.composition["A"] + self.composition["C"])
        return GCpercent
    def reverseComplement(self, sequence):
        #Takes the complement of the sequence and reverses it.
        reverseComplement = ""
        for base in sequence:
            if base == "A":
                reverseComplement = "T" + reverseComplement
            elif base == "T":
                reverseComplement = "A" + reverseComplement
            elif base == "G":
                reverseComplement = "C" + reverseComplement
            elif base == "C":
                reverseComplement = "G" + reverseComplement 
            else:
                print ("error")
        return (reverseComplement)
def reverseComplement(sequence):
    #Takes the complement of the sequence and reverses it.
    reverseComplement = ""
    for base in sequence:
        if base == "A":
            reverseComplement = "T" + reverseComplement 
        elif base == "T":
            reverseComplement = "A" + reverseComplement
        elif base == "G":
            reverseComplement = "C" + reverseComplement
        elif base == "C":
            reverseComplement = "G" + reverseComplement
        else:
            print ("error")
    return (reverseComplement)
def screenPrimers(insert, upstreamBackbone, downstreamBackbone):
    #run the main program to create and screen the forward and reverse primers and then add overhangs.
    forwardPrimer = screenPrimer(insert, "F")
    #print(forwardPrimer.sequence, "forward")
    reversePrimer = screenPrimer(insert, "R")
    #print(reversePrimer.sequence, "reverse")
    if len(forwardPrimer.sequence) and len(reversePrimer.sequence) != 0:
        primers = addOverhangs(forwardPrimer, reversePrimer, "reverse", upstreamBackbone, downstreamBackbone)
        return (primers[0].sequence, primers[1].sequence)
    else:
        return ("You cannot make primers with the given parameters", "You cannot make primers with the given parameters")
    
def screenPrimer(insert, whichPrimer, primerSize=18): #Make primers for an insert to amplify it such that it can be assembled with Gibson
    if primerSize < 40:
        #Make sure you are selecting the right region to prime. 
        #This algorithm creates primers that amplify the WHOLE SEQUENCE minus the stop codon at the end.
        noncanonicalStartCodon = False
        if whichPrimer == "F": 
            primer = DNASequence(insert.sequence[:primerSize])
            if (primer.sequence[:3].lower() != "atg"):
                primer = DNASequence(insert.sequence[3:primerSize])
                noncanonicalStartCodon = True
        elif whichPrimer == "R":
            primer = DNASequence(reverseComplement(insert.sequence[len(insert.sequence) - primerSize: len(insert.sequence)]))
        else:
            print("error")
        #Add various screens below here. You can update the parameters to be more or less stringent depending on how likely you would like your primers to work
        isGood = True
        if primer.Tm < 58.0 or primer.Tm > 59.5:
            isGood = False
        if float(primer.Homodimer.dg) < -10000.0:
            isGood = False
        if float(primer.Hairpin.dg) < -5000.0:
            isGood = False
    
        #return the shortest (18bp ≤) desireable primers
        if isGood == True:
            #screening returned a good primer; return the primer
            if noncanonicalStartCodon == True:
                primer = DNASequence("atg".upper() + primer.sequence)
            return primer
        else:
            #primer failed screening, try again with a larger primer using recursion
            if whichPrimer == "F":
                primer = screenPrimer(insert, "F", primerSize + 1) 
            elif whichPrimer == "R":
                primer = screenPrimer(insert, "R", primerSize + 1)
            return primer
    else:
        #parameters are too stringent; no primers were found that matched the criteria given. Return an empty string
        return DNASequence("")
def addOverhangs(primer1, primer2, orientation, leftBackbone, rightBackbone):
    #decide how the construct should be oriented in the final vector and add the overhangs so Gibson will add correctly.
    if orientation == "forward":
        primer2.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer2.sequence
        primer1.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer1.sequence
        return primer1, primer2
    elif orientation == "reverse":
        primer1.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer1.sequence
        primer2.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer2.sequence
        return primer1, primer2
    else:
        print ("please enter either a forward or reverse strand orientation")
def formatSeq(oldSeq):
    if not type(oldSeq) is str:
        oldSeq = ""
    new = oldSeq.replace("\n", "")
    for number in range(0,10):
        temp = new.replace(str(number), "")
        new = temp
    oldSeq = new
    new = oldSeq.replace(" ", "")
    return new.upper()

def appendPrimers(data, insert, tag, primerList, nameList):
    name = str(data.ix[number,"Gene"]).replace(" ", "") 
    nameFOR = name + " FOR " + tag + "-tagged"
    nameREV = name + " REV " + tag + "-tagged"
    primerList.append(screenPrimers(insert, leftbackbone , rightbackbone)[0])
    nameList.append(nameFOR)
    primerList.append(screenPrimers(insert, leftbackbone , rightbackbone)[1])
    nameList.append(nameREV)


primerList = []
nameList = []

for number in range(0,93):
    tag = ""
#We don't know whether unspecified proteins are N or C His-tagged, so we assume they could be either.
    leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())
    rightbackbone = DNASequence('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgcca'.upper())
    insert = DNASequence(formatSeq(data.ix[number,"Sequence - DNA sequence"]))
    insert = DNASequence(insert.sequence[:len(insert.sequence) - 3])
    if len(insert.sequence) != 0:
        tag = "C"
        appendPrimers(data, insert, tag, primerList, nameList)
    leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatc'.upper())
    rightbackbone = DNASequence('atggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccag'.upper())
    insert = DNASequence(formatSeq(data.ix[number,"Sequence - DNA sequence"]))
    if len(insert.sequence) != 0:
        tag = "N"
        appendPrimers(data, insert, tag, primerList, nameList)
        
dataframe = pandas.DataFrame(primerList, nameList)

writer = pandas.ExcelWriter('primerList2.xlsx', engine='openpyxl')

dataframe.to_excel(writer)

writer.save()

#Sequencing Primer Program

In [5]:
import primer3
import pandas
import math


def circularizePlasmid(plasmid):
    half = int(len(plasmid.sequence) / 2)
    #print(half)
    plasmid2 = DNASequence(plasmid.sequence[half:] + plasmid.sequence[:half])
    return (plasmid, plasmid2)

def findSeqPrimers(startPrimer, endPrimer, plasmid): #make sure insert is in middle of sequence for now
    plasmidToCheck = circularizePlasmid(plasmid)
    seqPrimerArr = []
    endPos = plasmidToCheck[0].sequence.find(reverseComplement(endPrimer.sequence)) - 500
    
    startPos = plasmidToCheck[0].sequence.find(startPrimer.sequence) + len(startPrimer.sequence)
    currentPos = startPos + 500
    while currentPos < endPos:
        seqPrimer = createSeqPrimer(currentPos, plasmidToCheck[0], 18, 0)
        seqPrimerArr.append(seqPrimer)
        if seqPrimer == "No primers found; please lower your search stringency":
            break
        currentPos = plasmidToCheck[0].sequence.find(seqPrimer) + len(seqPrimer) + 500
    
    return seqPrimerArr
        
def createSeqPrimer(currentPos, plasmid, primerSize, recursionNumber):
    #print(recursionNumber)
    if recursionNumber > 950:
        return "No primers found; please lower your search stringency"
    elif primerSize < 40:
        primer = DNASequence(plasmid.sequence[currentPos: currentPos + primerSize])
    
        isGood = True
        if primer.Tm < 58.0 or primer.Tm > 59.5:
            isGood = False
        if float(primer.Homodimer.dg) < -7000.0:
            isGood = False
        if float(primer.Hairpin.dg) < -500.0:
            isGood = False
    
            #return the shortest (18bp ≤) desireable primers
        if isGood == True:
            #screening returned a good primer; return the primer
            return primer.sequence
        else:
            #primer failed screening, try again with a larger primer using recursion
            return createSeqPrimer(currentPos, plasmid, primerSize + 1, recursionNumber + 1)
    else:
        primerSize = 18
        return createSeqPrimer(currentPos - 1, plasmid, primerSize, recursionNumber + 1)

forward = DNASequence('tccggatatagttcctcctttcagc'.upper())
reverse = DNASequence('GGTGATGTCGGCGATATAGGC'.upper())
plas = DNASequence('cgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgccaatccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatcttaAGCAATGGTACGTTGGATCTCGATGATTTCGAATACTTCGATCACATCGCCAGTGCGGACGTCGTTGTAGTTCTTAACGCCGATACCACATTCCATACCGTTACGGACTTCGTTAACGTCATCTTTGAAGCGGCGCAGGGACTCCAGCTCGCCTTCGTAGATAACCACGTTGTCACGCAGAACGCGGATCGGGTTGTGACGTTTAACCACACCTTCGGTAACCATACAGCCTGCGATGGCACCAAATTTCGGCGATTTGAACACGTCACGAACTTCCGCCAGACCGATAATCTGCTGTTTCAGTTCCGGAGACAGCATACCGCTCATCGCCGCTTTCACTTCGTCAATCAGGTTATAGATGACGGAGTAGTAACGCAGATCCAGGCTTTCCGCTTCAATCACTTTACGTGCAGAGGCATCAGCACGTACGTTAAAGCCAACCAGGATGGCGTTGGACGCCGCAGCCAGGGTGGCGTCGGTTTCGGTGATACCACCTACGCCAGAACCGATGATCTTCACTTTAACTTCGTCAGTAGACAGTTTCAGCAAGGAGTCGGAGATCGCTTCGACAGAACCCTGTACGTCTGCCTTCAGGACGATATTCACTTCGTGAACTTCGCCTTCGGTCATGTTGGCGAACATGTTCTCGAGTTTAGATTTCTGCTGACGCGCCAGTTTAACTTCGCGGAATTTACCCTGACGATAGAGTGCAACTTCACGCGCTTTCTTCTCGTCACGTACAACGGTAACTTCATCACCCGCAGCCGGTACGCCGGACAGGCCGAGGATTTCCACCGGAATGGACGGACCCGCTTCCAGCACTTCCTGACCCAGTTCGTTACGCATCGCACGAACACGACCGTATTCGAAGCCACACAGAACGATATCGCCCTTGTGCAGAGTACCTTCACGTACCAGAACGGTAGCAACCGGACCACGACCTTTATCGAGGAAGGATTCGATAACCGCACCGCTCGCCATACCTTTACGTACCGCTTTCAGCTCCAGAACTTCCGCCTGCAGCAGGATAGCGTCCAGCAGTTCATCGATACCGGTACCCGCTTTCGCAGATACGTGTACGAACTGGCTTTCACCGCCCCACTCTTCCGGCAGGATGCCGTACTGGGAGAGTTCGTTCTTAACGCGATCCGGATCAGCTTCTGGTTTATCGATCTTGTTCACTGCAACCACCACCGGTACCTGCGCCGCTTTCGCGTGCTGGATTGCTTCGATGGTCTGCGGCATCACACCGTCGTCGGCAGCAACAACCAGGACTACGATGTCCGTTGCCTGCGCACCACGAGCACGCATTGAAGTAAACGCGGCGTGCCCCGGGGTGTCCAGGAAGGTGATCATGCCGTTTTCAGTTTCAACGTGGTATGCACCAATGTGCTGGGTAATGCCGCCCGCTTCGCCAGAGGCCACTTTCGTTGAACGAATGTAGTCCAGCAGAGAGGTTTTACCGTGGTCAACGTGACCCATGATGGTCACAACCGGCGCGCGCGGTTCAGCCGCAGCACCCGTGTCACGGTCGCTCATTACCGCCTCTTCCAGCTCGTTTTCACGACGCAGGATAACTTTATGGCCCATCTCTTCAGCAACCAGCTGTGCGGTTTCCTGATCGATAACCTGGTTGATGGTTGCCATTGCGCCCAGTTTCATCATCGCTTTGATGACCTGAGAGCCTTTAACCGCCATCTTGTTCGCCAGTTCGCCAACGGTGATAGTTTCGCCGATCACAACGTCACGGTTAACGGCCTGAGCAGGCTTCTGGAAGCCTTGCTGCAGCGAAGAACCTTTACGTTTTCCGCCTTTACCGCCACGTACTGCTGCGCGTGCTTCTTCACGATCAGCTTTTGATTCAGCGTGTTTGTTGCCTTTCTTCGGACGCGCTGCTTTCGCGTTACGACCACGGCCACGGCCGCCTTCGACTTCACGATCGCTTTCGTCTTCTGCCTGGCGAGCATGTTGAGAAGTAGTGACGTGATAATCGCTGGAATCTTCAGTCGGTTCCGCGTTATCAGTCCATTTGTTTTCTTCCGCCATACGACGTGCTTCTTCAGCAACGCGACGTGCTTCTTCTTCGAGTTTACGACGCGCTTCTTCTTCAGCTTTACGCTTGAGCTCTGCAGCTTCCTGCTCACGGCGGGCTTTTTCAGCCTGGGCGTTTTTAGTCATATCGTCTTGTTGATTGCTCACTTTGTCTTTTTCCGCAGCTTCACGTTTCGCTTGTTCAGCAGCTTCACGCTTAGCTTGTTCTGCGGCCTCACGTTCAGCTTTTTGTTGCGCCTCGCGTTTAGCCGATTCTTCTGCCTCACGACGGGCTTGCTCTTCCGCTTCACGCTGCGCTTGCTCTTCCGCTGCAAGGCGTTCAGCCTCTTGCGGATCGCGTTTCACAAAGGTGCGTTTCTTGCGGACTTCGATTTGTACCGATTTGCTTTTTCCACCGGTACCAGGAATGTTAAGGGTGCTGCGTGTTTTACGTTGCAGCGTCAATTTGTCCGGGCCTGAATTTTTCTGATTCAGGTGGTCAATCAAAGTCTGTTTCTCTTGTGCAGACACAGAGTCGTCAGCAGACTTCCGGATACCTGCATCAGCAAATTGCTGTACCAGGCGTTCCACGGAGGTCTGTCGCTCTGCGGCCAGCGTTTTAATCGTTACATCTGTcatatggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcg'.upper())

beta = findSeqPrimers(forward, reverse, plas)
for alpha in beta:
    print(alpha)

TTCCGCTTCAATCACTTTACGTGCAG
AGAGTACCTTCACGTACCAGAACGGTA
CCAGAGGCCACTTTCGTTGAACG
GATCGCTTTCGTCTTCTGCCTGG


In [12]:
from benchlingapi import BenchlingAPI

bench_api_key = 'sk_gnGGnlq0laAcbBcI9yVEIqpE7opJQ'
benchlingapi = BenchlingAPI(bench_api_key)

print(benchlingapi.find_folder('Program Generated Vectors', query='name', regex=True))

an = {"color": "#303030", "end": 25, "name": "Untitled", "start": 16, "strand": 1, "type": "gene"}

#print(benchlingapi.create_sequence('testSeqProgram', 'cagggcagccgg', True, 'lib_f0TfvHp8', description='this is a test sequence', annotations=[an]))


import requests

#resp = requests.get('https://api.benchling.com/v1/sequences', auth=(bench_api_key, ''))
#if resp.status_code != 200:
    # This means something went wrong.
    #raise ApiError('GET /tasks/ {}'.format(resp.status_code))

#print(resp.json()[0])
# for todo_item in resp.json():
#     print('{} {}'.format(todo_item['id'], todo_item['summary']))


{'count': 0, 'created_at': '2017-08-01T07:03:23.289667+00:00', 'description': 'Sequences generated from the primer program inputted into the vector are available here', 'id': 'lib_iP53B30G', 'modified_at': '2017-08-01T07:03:23.289667+00:00', 'name': 'Program Generated Vectors', 'owner': 'ent_GopUD1my', 'permissions': {'admin': True, 'appendable': True, 'owner': False, 'readable': True, 'writable': True}, 'sequences': [], 'type': 'INVENTORY'}


In [70]:
#resp.json()['sequences'][0]

seqPrimers = []
seqPrimerNames = []

def unpackPrimers(nameList, seqList, finNameList, finSeqList):
    for names in nameList:
        finNameList.append(names)
    for primers in seqList:
        finSeqList.append(primers)

a = createVector(DNASequence('atgGCAGCTAAAGACGTAAAATTCGGTAACGACGCTCGTGTGAAAATGCTGCGCGGCGTAAACGTACTGGCAGATGCAGTGAAAGTTACCCTCGGTCCAAAAGGCCGTAACGTAGTTCTGGATAAATCTTTCGGTGCACCGACCATCACCAAAGATGGTGTTTCCGTTGCTCGTGAAATCGAACTGGAAGACAAGTTCGAAAATATGGGTGCGCAGATGGTGAAAGAAGTTGCCTCTAAAGCAAACGACGCTGCAGGCGACGGTACCACCACTGCAACCGTACTGGCTCAGGCTATCATCACTGAAGGTCTGAAAGCTGTTGCTGCGGGCATGAACCCGATGGACCTGAAACGTGGTATCGACAAAGCGGTTACCGCTGCAGTTGAAGAACTGAAAGCGCTGTCCGTACCATGCTCTGACTCTAAAGCGATTGCTCAGGTTGGTACCATCTCCGCTAACTCCGACGAAACCGTAGGTAAACTGATCGCTGAAGCGATGGACAAAGTCGGTAAAGAAGGCGTTATCACCGTTGAAGACGGTACCGGTCTGCAGGACGAACTGGACGTGGTTGAAGGTATGCAGTTCGACCGTGGCTACCTGTCTCCTTACTTCATCAACAAGCCGGAAACTGGCGCAGTAGAACTGGAAAGCCCGTTCATCCTGCTGGCTGACAAGAAAATCTCCAACATCCGCGAAATGCTGCCGGTTCTGGAAGCTGTTGCCAAAGCAGGCAAACCGCTGCTGATCATCGCTGAAGATGTAGAAGGCGAAGCGCTGGCAACTCTGGTTGTTAACACCATGCGTGGCATCGTGAAAGTCGCTGCGGTTAAAGCACCGGGCTTCGGCGATCGTCGTAAAGCTATGCTGCAGGATATCGCAACCCTGACTGGCGGTACCGTGATCTCTGAAGAGATCGGTATGGAGCTGGAAAAAGCAACCCTGGAAGACCTGGGTCAGGCTAAACGTGTTGTGATCAACAAAGACACCACCACTATCATCGATGGCGTGGGTGAAGAAGCTGCAATCCAGGGCCGTGTTGCTCAGATCCGTCAGCAGATTGAAGAAGCAACTTCTGACTACGACCGTGAAAAACTGCAGGAACGCGTAGCGAAACTGGCAGGCGGCGTTGCAGTTATCAAAGTGGGTGCTGCTACCGAAGTTGAAATGAAAGAGAAAAAAGCACGCGTTGAAGATGCCCTGCACGCGACCCGTGCTGCGGTAGAAGAAGGCGTGGTTGCTGGTGGTGGTGTTGCGCTGATCCGCGTAGCGTCTAAACTGGCTGACCTGCGTGGTCAGAACGAAGACCAGAACGTGGGTATCAAAGTTGCACTGCGTGCAATGGAAGCTCCGCTGCGTCAGATCGTATTGAACTGCGGCGAAGAACCGTCTGTTGTTGCTAACACCGTTAAAGGCGGCGACGGCAACTACGGTTACAACGCAGCAACCGAAGAATACGGCAACATGATCGACATGGGTATCCTGGATCCAACCAAAGTAACTCGTTCTGCTCTGCAGTACGCAGCTTCTGTGGCTGGCCTGATGATCACCACCGAATGCATGGTTACCGACCTGCCGAAAAACGATGCAGCTGACTTAGGCGCTGCTGGCGGTATGGGCGGCATGGGTGGCATGGGCGGCATGATGtaa'.upper()), 'N', "groL")
b = createVector(DNASequence('atgAATATTCGTCCATTGCATGATCGCGTGATCGTCAAGCGTAAAGAAGTTGAAACTAAATCTGCTGGCGGCATCGTTCTGACCGGCTCTGCAGCGGCTAAATCCACCCGCGGCGAAGTGCTGGCTGTCGGCAATGGCCGTATCCTTGAAAATGGCGAAGTGAAGCCGCTGGATGTGAAAGTTGGCGACATCGTTATTTTCAACGATGGCTACGGTGTGAAATCTGAGAAGATCGACAATGAAGAAGTGTTGATCATGTCCGAAAGCGACATTCTGGCAATTGTTGAAGCGtaa'.upper()), 'N', "groS")
c = createVector(DNASequence('atgACTCAAGTCGCGAAGAAAATTCTGGTGACGTGCGCACTGCCGTACGCTAACGGCTCAATCCACCTCGGCCATATGCTGGAGCACATCCAGGCTGATGTCTGGGTCCGTTACCAGCGAATGCGCGGCCACGAGGTCAACTTCATCTGCGCCGACGATGCCCACGGTACACCGATCATGCTGAAAGCTCAGCAGCTTGGTATCACCCCGGAGCAGATGATTGGCGAAATGAGTCAGGAGCATCAGACTGATTTCGCAGGCTTTAACATCAGCTATGACAACTATCACTCGACGCACAGCGAAGAGAACCGCCAGTTGTCAGAACTTATCTACTCTCGCCTGAAAGAAAACGGTTTTATTAAAAACCGCACCATCTCTCAGCTGTACGATCCGGAAAAAGGCATGTTCCTGCCGGACCGTTTTGTGAAAGGCACCTGCCCGAAATGTAAATCCCCGGATCAATACGGCGATAACTGCGAAGTCTGCGGCGCGACCTACAGCCCGACTGAACTGATCGAGCCGAAATCGGTGGTTTCTGGCGCTACGCCGGTAATGCGTGATTCTGAACACTTCTTCTTTGATCTGCCCTCTTTCAGCGAAATGTTGCAGGCATGGACCCGCAGCGGTGCGTTGCAGGAGCAGGTGGCAAATAAAATGCAGGAGTGGTTTGAATCTGGCCTGCAACAGTGGGATATCTCCCGCGACGCCCCTTACTTCGGTTTTGAAATTCCGAACGCGCCGGGCAAATATTTCTACGTCTGGCTGGACGCACCGATTGGCTACATGGGTTCTTTCAAGAATCTGTGCGACAAGCGCGGCGACAGCGTAAGCTTCGATGAATACTGGAAGAAAGACTCCACCGCCGAGCTGTACCACTTCATCGGTAAAGATATTGTTTACTTCCACAGCCTGTTCTGGCCTGCCATGCTGGAAGGCAGCAACTTCCGCAAGCCGTCCAACCTGTTTGTTCATGGCTATGTGACGGTGAACGGCGCAAAGATGTCCAAGTCTCGCGGCACCTTTATTAAAGCCAGCACCTGGCTGAATCATTTTGACGCAGACAGCCTGCGTTACTACTACACTGCGAAACTCTCTTCGCGCATTGATGATATCGATCTCAACCTGGAAGATTTCGTTCAGCGTGTGAATGCCGATATCGTTAACAAAGTGGTTAACCTGGCCTCCCGTAATGCGGGCTTTATCAACAAGCGTTTTGACGGCGTGCTGGCAAGCGAACTGGCTGACCCGCAGTTGTACAAAACCTTCACTGATGCCGCTGAAGTGATTGGTGAAGCGTGGGAAAGCCGTGAATTTGGTAAAGCCGTGCGCGAAATCATGGCGCTGGCTGATCTGGCTAACCGCTATGTCGATGAACAGGCTCCGTGGGTGGTGGCGAAACAGGAAGGCCGCGATGCCGACCTGCAGGCAATTTGCTCAATGGGCATCAACCTGTTCCGCGTGCTGATGACTTACCTGAAGCCGGTACTGCCGAAACTGACCGAGCGTGCAGAAGCATTCCTCAATACGGAACTGACCTGGGATGGTATCCAGCAACCGCTGCTGGGCCACAAAGTGAATCCGTTCAAGGCGCTGTATAACCGCATCGATATGAGGCAGGTTGAAGCACTGGTGGAAGCCTCTAAAGAAGAAGTAAAAGCCGCTGCCGCGCCGGTAACTGGCCCGCTGGCAGATGATCCGATTCAGGAAACCATCACCTTTGACGACTTCGCTAAAGTTGACCTGCGCGTGGCGCTGATTGAAAACGCAGAGTTTGTTGAAGGTTCTGACAAACTGCTGCGCCTGACGCTGGATCTCGGCGGTGAAAAACGCAATGTCTTCTCCGGTATTCGTTCTGCTTACCCGGATCCGCAGGCACTGATTGGTCGTCACACCATTATGGTGGCTAACCTGGCACCACGTAAAATGCGCTTCGGTATCTCTGAAGGCATGGTGATGGCTGCCGGTCCTGGCGGGAAAGATATTTTCCTGCTAAGCCCGGATGCCGGTGCTAAACCGGGTCATCAGGTGAAAtaa'.upper()), "C", "Met")
                 
unpackPrimers(a[0], a[1], seqPrimerNames, seqPrimers)
unpackPrimers(b[0], b[1], seqPrimerNames, seqPrimers)
unpackPrimers(c[0], c[1], seqPrimerNames, seqPrimers)

df = pd.DataFrame(seqPrimers, seqPrimerNames)
print(df)

writer = pd.ExcelWriter('test3.xlsx', engine='openpyxl')

df.to_excel(writer)

writer.save()

<Response [201]>
<Response [201]>
<Response [201]>
                                                      0
seqFOR1 groL N-tagged              CAGCGCAACACCACCACCAG
seqFOR2 groL N-tagged               TCAGCAGCGGTTTGCCTGC
seqFOR1 Met C-tagged       AGCGGCTTTTACTTCTTCTTTAGAGGCT
seqFOR2 Met C-tagged   CGAAATCTTCCAGGTTGAGATCGATATCATCA
seqFOR3 Met C-tagged        CATTTCGCTGAAAGAGGGCAGATCAAA


In [2]:
import requests
import pandas as pd
import numpy as np

def createVector(insert, tag, name):
    seqPrimerList = []
    seqPrimerNameList = []
    insert = insert.sequence
    annotations = []
    pBR322_origin = "cgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatct".upper()
    kanR2 = "atgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaa".upper()
    rop = "gtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctga".upper()
    t7Terminator = "atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcag".upper()
    tet300 = "gccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagag".upper()
    pGEX_3_primer = "cctctgacacatgcagctcccgg".upper()
    thrombin = thrombin = "gctgccgcgcggcaccag".upper()
    t7promoter = "cctatagtgagtcgtatta".upper()
    lacO = "ggggaattgttatccgctcacaattccc".upper()
    hisTag = "gtgatgatgatgatgatg".upper()
    seqRev = "gcctatatcgccgacatcacc".upper()
    seqFor = "tccggatatagttcctcctttcagc".upper()
    rbs = "ctccttcttaaagttaa".upper()
    hisPlusStop = "tcagtggtggtggtggtggtg".upper()
    lacI = "gtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtga".upper()
    tet576 = "cccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatg".upper()
    
    
    if tag =="C":
        leftBackbone = "cgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgccaatccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg".upper()
        rightBackbone = "ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcaggg".upper()
        sequence = leftBackbone + reverseComplement(insert[:len(insert) - 3]) + rightBackbone
        endLen = len(leftBackbone) + len(insert) - 3
        addAnnotation(annotations, reverseComplement(insert[:len(insert) - 3]), sequence, name, 'gene', "#0000ff")
        
    if tag =="N":
        leftBackbone = "cgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgccaatccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatc".upper()
        rightBackbone = "atggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcg".upper()
        sequence = leftBackbone + reverseComplement(insert) + rightBackbone
        endLen = len(leftBackbone) + len(insert)
        addAnnotation(annotations, reverseComplement(insert), sequence, name, 'gene', "#0000ff")
        
    offName = str(name) + " " + tag + "-tagged"
    
    beta = findSeqPrimers(DNASequence(seqFor), DNASequence(reverseComplement(seqRev)), DNASequence(sequence))
    count = 0
    for alpha in beta:
        count += 1
        nom = "seqFOR" + str(count) + " " + offName
        addAnnotation(annotations, alpha, sequence, nom, "primer", "#f2c22e")
        seqPrimerNameList.append(nom)
        seqPrimerList.append(alpha)
    
    addAnnotation(annotations, pBR322_origin, sequence, "pBR322 Origin", "origin", "#9faa9a")
    addAnnotation(annotations, kanR2, sequence, "Kan R2", "gene", "#359797")
    addAnnotation(annotations, t7Terminator, sequence, "T7 Terminator", "terminator", "#d24b19")
    addAnnotation(annotations, tet300, sequence, "tet (300-563)", "gene", "#ffe2b9")
    addAnnotation(annotations, pGEX_3_primer, sequence, "pGEX_3_primer", "primer", "#5b637e")
    addAnnotation(annotations, thrombin, sequence, "thrombin", "tag", "#5f631d")
    addAnnotation(annotations, t7promoter, sequence, "T7 Promoter", "promoter", "#c8498b")
    addAnnotation(annotations, lacO, sequence, "lacO", "regulatory", "#fffbaa")
    addAnnotation(annotations, lacO, sequence, "lacI", "regulatory", "#c3dff3")
    addAnnotation(annotations, hisTag, sequence, "His Tag", "tag", "#f9c5fe")
    addAnnotation(annotations, seqFor, sequence, "pET28+ Forward Primer", "primer", "#fffafa")
    addAnnotation(annotations, seqRev, sequence, "pET28+ Reverse Primer", "primer", "#fffafa")
    addAnnotation(annotations, rbs, sequence, "T7 Ribosome Binding Site", "feature", "#e12222")
    addAnnotation(annotations, hisPlusStop, sequence, "His Tag and Stop Codon", "feature", "#ddb34e")
    addAnnotation(annotations, tet576, sequence, "tet(576-851)", "feature", "#ba9231")
    addAnnotation(annotations, rop, sequence, "ROP", "feature", "#1ebdac")
    
    
    
    data = {
      "aliases": [],
      "annotations": annotations,
      "bases": sequence,
      "circular": True,
      "description": "program generated",
      "folder": "lib_iP53B30G",
      "name": offName,
      "tags": []
    }

    key = 'sk_gnGGnlq0laAcbBcI9yVEIqpE7opJQ'
    resp = requests.post('https://api.benchling.com/v1/sequences/', json=data, auth=(key, ''))
    print(resp)
    return (seqPrimerNameList, seqPrimerList)

def findSequence(sequence, plasmid):
    if sequence in plasmid:
        a = plasmid.find(sequence)
        #print(a, a + len(sequence))
    else:
        return (-1, -1)
    return (a, a + len(sequence))
def addAnnotation(annotations, sequence, plasmid, name, annotationType, color):
    annotation = {
          "color": color,
          "end": findSequence(sequence, plasmid)[1],
          "name": name,
          "start": findSequence(sequence, plasmid)[0],
          "strand": 0,
          "type": annotationType
    }
    if findSequence(sequence, plasmid) != (-1, -1):
        annotations.append(annotation)
    return annotations
    
#createVector('ATGGAAAAGACATATAACCCACAAGATATCGAACAGCCGCTTTACGAGCACTGGGAAAAGCAGGGCTACTTTAAGCCTAATGGCGATGAAAGCCAGGAAAGTTTCTGCATCATGATCCCGCCGCCGAACGTCACCGGCAGTTTGCATATGGGTCACGCCTTCCAGCAAACCATCATGGATACCATGATCCGCTATCAGCGCATGCAGGGCAAAAACACCCTGTGGCAGGTCGGTACTGACCACGCCGGGATCGCTACCCAGATGGTCGTTGAGCGCAAGATTGCCGCAGAAGAAGGTAAAACCCGTCACGACTACGGCCGCGAAGCTTTCATCGACAAAATCTGGGAATGGAAAGCGGAATCTGGCGGCACCATTACCCGTCAGATGCGCCGTCTCGGCAACTCCGTCGACTGGGAGCGTGAACGCTTCACCATGGACGAAGGCCTGTCCAATGCGGTGAAAGAAGTTTTCGTTCGTCTGTATAAAGAAGACCTGATTTACCGTGGCAAACGCCTGGTAAACTGGGATCCGAAACTGCGCACCGCTATCTCTGACCTGGAAGTCGAAAACCGCGAATCGAAAGGTTCGATGTGGCACATCCGCTATCCGCTGGCTGACGGTGCGAAAACCGCAGACGGTAAAGATTATCTGGTGGTCGCGACTACCCGTCCAGAAACCCTGCTGGGCGATACTGGCGTAGCCGTTAACCCGGAAGATCCGCGTTACAAAGATCTGATTGGCAAATATGTCATTCTGCCGCTGGTTAACCGTCGTATTCCGATCGTTGGCGACGAACACGCCGACATGGAAAAAGGCACCGGCTGCGTGAAAATCACTCCGGCGCACGACTTTAACGACTATGAAGTGGGTAAACGTCACGCCCTGCCGATGATCAACATCCTGACCTTTGACGGCGATATCCGTGAAAGCGCCCAGGTGTTCGATACCAAAGGTAACGAATCTGACGTTTATTCCAGCGAAATCCCTGCAGAGTTCCAGAAACTGGAGCGTTTTGCTGCACGTAAAGCAGTCGTTGCCGCAGTTGACGCGCTTGGCCTGCTGGAAGAAATTAAACCGCACGACCTGACCGTTCCTTACGGCGACCGTGGCGGCGTAGTTATCGAACCAATGCTGACCGACCAGTGGTACGTGCGTGCCGATGTCCTGGCGAAACCGGCGGTTGAAGCGGTTGAGAACGGCGACATTCAGTTCGTACCGAAGCAGTACGAAAACATGTACTTCTCCTGGATGCGCGATATTCAGGACTGGTGTATCTCTCGTCAGTTGTGGTGGGGTCACCGTATCCCGGCATGGTATGACGAAGCGGGTAACGTTTATGTTGGCCGCAACGAAGACGAAGTGCGTAAAGAAAATAACCTCGGTGCTGATGTTGTCCTGCGTCAGGACGAAGACGTTCTCGATACCTGGTTCTCTTCTGCGCTGTGGACCTTCTCTACCCTTGGCTGGCCGGAAAATACCGACGCCCTGCGTCAGTTCCACCCAACCAGCGTGATGGTATCTGGTTTCGACATCATTTTCTTCTGGATTGCCCGCATGATCATGATGACCATGCACTTCATCAAAGATGAAAATGGCAAACCGCAGGTGCCGTTCCACACCGTTTACATGACCGGCCTGATTCGTGATGACGAAGGCCAGAAGATGTCCAAATCCAAGGGTAACGTTATCGACCCACTGGATATGGTTGACGGTATTTCGCTGCCAGAACTGCTGGAAAAACGTACCGGCAATATGATGCAGCCGCAGCTGGCGGACAAAATCCGTAAGCGCACCGAGAAGCAGTTCCCGAACGGTATTGAGCCGCACGGTACTGACGCGCTGCGCTTCACCCTGGCGGCGCTGGCGTCTACCGGTCGTGACATCAACTGGGATATGAAGCGTCTGGAAGGTTACCGTAACTTCTGTAACAAGCTGTGGAACGCCAGCCGCTTTGTGCTGATGAACACAGAAGGTCAGGATTGCGGCTTCAACGGCGGCGAAATGACGCTGTCGCTGGCGGACCGCTGGATTCTGGCGGAGTTCAACCAGACCATCAAAGCGTACCGCGAAGCGCTGGACAGCTTCCGCTTCGATATCGCCGCAGGCATTCTGTATGAGTTCACCTGGAACCAGTTCTGTGACTGGTATCTCGAGCTGACCAAGCCGGTAATGAACGGTGGCACCGAAGCAGAACTGCGCGGTACTCGCCATACGCTGGTGACTGTACTGGAAGGTCTGCTGCGCCTCGCGCATCCGATCATTCCGTTCATCACCGAAACCATCTGGCAGCGTGTGAAAGTACTTTGCGGTATCACTGCCGACACCATCATGCTGCAGCCGTTCCCGCAGTACGATGCATCTCAGGTTGATGAAGCCGCACTGGCCGACACCGAATGGCTGAAACAGGCGATCGTTGCGGTACGTAACATCCGTGCAGAAATGAACATCGCGCCGGGCAAACCGCTGGAGCTGCTGCTGCGTGGTTGCAGCGCGGATGCAGAACGTCGCGTAAATGAAAACCGTGGCTTCCTGCAAACCCTGGCGCGTCTGGAAAGTATCACCGTGCTGCCTGCCGATGACAAAGGTCCGGTTTCCGTTACGAAGATCATCGACGGTGCAGAGCTGCTGATCCCGATGGCTGGCCTCATCAACAAAGAAGATGAGCTGGCGCGTCTGGCGAAAGAAGTGGCGAAGATTGAAGGTGAAATCAGCCGTATCGAGAACAAACTGGCGAACGAAGGCTTTGTCGCCCGCGCACCGGAAGCGGTCATCGCGAAAGAGCGTGAGAAGCTGGAAGGCTATGCGGAAGCGAAAGCGAAACTGATTGAACAGCAGGCTGTTATCGCCGCGCTGtaa'.upper(), 'C')